# Exploration

In [ ]:
PATH_TREEBANK = "../data/sud_naija-NSC.with_prediction.conllu"

In [ ]:
# le fichier est t-il bien présent ?
try:
    with open(PATH_TREEBANK, "r") as f:
        print("Le fichier est présent au chemin : {}".format(PATH_TREEBANK))
        pass
except FileNotFoundError:
    print("Le fichier n'est PAS , revoyez le chemin : {}".format(PATH_TREEBANK))
    exit()

In [ ]:
# On peut commencer à parser le fichier avec la librairie conllup
from conllup.conllup import readConlluFile 
sentences = readConlluFile(PATH_TREEBANK)

if len(sentences) == 0:
    raise ValueError("Le fichier est vide !")
else :
    print("Le fichier contient {} phrases".format(len(sentences)))

In [ ]:
# A quoi ressemble notre objet "sentence"
import json
print(json.dumps(sentences[0], indent=4)) 

Nous pouvons retrouver, dans `sentence["metaJson"]`, les meta-informations des phrases du conllu :
- `sent_id`: identifiant de la phrase dans le treebank.
- `sound_url`: lien url de l'audio de la phrase (ce treebank NaijaSynCor est un corpus oral) .
- `speaker_id`: identifiant (anonymisé) du locuteur.
- `text`: version "prosodique" du texte. Les ponctuations sont plus riches et des marqueurs prosodiques (retranscrivant l'oral) sont ajoutés.
- `text_en`: traduction anglaise.
- `text_ortho`: version littéraire du texte.
- `timestamp`: date de la dernière annotation.


In [ ]:
# voici donc ces meta-données seulement
print(json.dumps(sentences[0]['metaJson'], indent=4)) 

In [ ]:
# Comment accéder aux tokens ?
# ils sont imbriqués dans sentence['treeJson']['nodesJson']
sentence_tokens = sentences[0]['treeJson']['nodesJson']
print(json.dumps(sentence_tokens, indent=4))


In [ ]:
# Les tokens sont donc dans un dictionnaire avec comme clé l'ID du token 
# et comme valeur un dictionnaire avec les informations du token

# Regardons le premier token
print(json.dumps(sentence_tokens['1'], indent=4))

Voici la composition d'un token au format json (voir [spécifications officielles ici](https://universaldependencies.org/format.html) )
- `ID`: Identifiant du token dans la phrase,
- `FORM`: Forme du token,
- `LEMMA`: Lemme du token,
- `UPOS`: Catégorie du discours universelle (Universal Part Of Speech)
- `XPOS`: Catégorie du discours **optionnelle**
- `FEATS`: Liste de features morphologiques de l'inventaire universel ou d'une extension spécifique à la langue
- `HEAD`: Identifiant du gouverneur du token (0 si le token est la tête de la phrase)
- `DEPREL`: Relation de dépendance avec le gouverneur (`root` si le token est la tête de la phrase)
- `DEPS`: Graphes de dépendances améliorés sous la forme d'une liste de paires tête-déprel 
- `MISC`: Toutes les autres données non universelles peuvent être insérées dans cet objet MISC. Voici quelques exemples de valeur misc
    - `AlignBegin`: alignement temporelle **du début** du token dans l'audio de la phrase (en milliseconde)
    - `AlignEnd`: alignement temporelle de **la fin** du token dans l'audio de la phrase (en milliseconde),
    - `Gloss`: glose anglaise
    - `upos_pred`: Catégorie du dicours **prédit**
    - `head_pred`: Identifiant **prédit** du gouverneur,
    - `deprel_pred`: Relation de dépendance **prédite**,


In [ ]:
# Maintenant qu'on connait la structure, on peut compter le nombre de tokens dans tous le corpus
nb_tokens = 0
for sentence in sentences:
    nb_tokens += len(sentence['treeJson']['nodesJson'])
print("Le corpus contient {} tokens".format(nb_tokens))



In [ ]:
# calculons le nombre de tokens par phrase en moyenne dans ce corpus
nb_tokens = 0
for sentence in sentences:
    nb_tokens += len(sentence['treeJson']['nodesJson'])
print("Le corpus contient {} tokens".format(nb_tokens))
print("Le corpus contient {} phrases".format(len(sentences)))
print("Le nombre moyen de tokens par phrase est de {}".format(nb_tokens/len(sentences)))

## Visualisation
Regardons nos données via des graphes.

In [ ]:
# Nous utilisons matplotlib pour faire faire nos graphiques
import matplotlib as mpl
import matplotlib.pyplot as plt

# Voici des paramètres pour que les graphiques soient plus lisibles, changez les à votre convenance
params = {
   'axes.labelsize': 8,
 #  'text.fontsize': 8,
   'legend.fontsize': 10,
   'xtick.labelsize': 10,
   'ytick.labelsize': 10,
   'text.usetex': False,
   'figure.figsize': [10, 5]
   }
mpl.rcParams.update(params)
plt.style.use('seaborn-v0_8-darkgrid') # pour avoir un fond gris quadrillé

In [ ]:
# Affichons la distribution des tokens par phrase

## Recupérons le nombre de tokens par phrase dans une liste
tokens_per_sentence = []
for sentence in sentences:
    tokens_per_sentence.append(len(sentence['treeJson']['nodesJson']))

## Traçons l'histogramme
plt.hist(tokens_per_sentence, bins=range(0, 60), edgecolor='white') 
plt.title("Distribution du nombre de tokens par phrase (ajusté)") # ajoute un titre au graphique
plt.xlabel("Nombre de tokens") # ajoute un titre à l'axe des abscisses
plt.ylabel("Nombre de phrases") # ajoute un titre à l'axe des ordonnées


## Optionel : Ajoutons une ligne verticale rouge pour la moyenne
length_mean = nb_tokens/len(sentences)
plt.axvline(x=length_mean, color='red', linestyle='dashed', linewidth=1) # place la ligne verticale de moyenne (en rouge)
plt.text(length_mean*1.1, plt.ylim()[1]*0.9, 'Moyenne = {:.2f}'.format(length_mean)) # place la légende de la ligne verticale


## Affichons le graphique
plt.show()

In [ ]:
# Faisons une fonction "counter" qui, depuis une liste d'étiquettes, retourne un dictionnaire avec le nombre d'occurences de chaque étiquette
# le compteur doit être ordonné par ordre alphabétique
def make_counter(labels):
    counter = {}
    for label in labels:
        if label in counter:
            counter[label] += 1
        else:
            counter[label] = 1
    # sort alphabetically before return
    counter = {k: v for k, v in sorted(counter.items(), key=lambda item: item[0])}
    return counter




# testons sur une liste exemple
labels = ["VERB", "NOUN", "NOUN", "VERB", "VERB"]
print(make_counter(labels))

In [ ]:
# Affichons la distribution des différentes catégories du discours (UPOS)

## Recupérons le nombre de tokens par phrase dans une liste
upos = []
for sentence in sentences:
    for token in sentence['treeJson']['nodesJson'].values():
        upos.append(token['UPOS'])

upos_counter = make_counter(upos)
print(json.dumps(upos_counter, indent=4))


In [ ]:
## Traçons l'histogramme

plt.bar(upos_counter.keys(), upos_counter.values())
plt.xticks(rotation=45) # pour que les étiquettes soient lisibles (on les tourne de 45°)

Les ponctuations sont grandement dominantes, il faudra faire attention à ce que ça ne biaise pas les évaluations du parseur (il est généralement plus facile de faire des prédictions sur les ponctuations)

## Distance et ordre syntaxique

In [ ]:
# Le naija est t'il une langue à tête initiale ou finale ?
# Pour cela, nous allons compter le nombre de dépendances allant vers la gauche et vers la droite
left_n = 0
right_n = 0
for sentence in sentences:
    for token in sentence['treeJson']['nodesJson'].values():
        if token['ID'] == '0':
            # On ne compte pas les racines
            continue

        if token['HEAD'] < int(token['ID']):
            right_n += 1
        else:
            left_n += 1
print("Il y a {} dépendances vers la gauche et {} vers la droite".format(left_n, right_n))


In [ ]:
# Est-ce que ces directions sont différentes selon les catégories du discours ?
# Pour cela, nous allons compter le nombre de dépendances allant vers la gauche et vers la droite pour chaque catégorie du discours
# Nous allons utiliser un dictionnaire de dictionnaires pour stocker ces informations
# Le premier niveau de clé sera la catégorie du discours
# Le second niveau de clé sera la direction de la dépendance (left ou right)
# La valeur sera le nombre de dépendances dans cette catégorie et dans cette direction

upos_directions = {}
for sentence in sentences:
    for token in sentence['treeJson']['nodesJson'].values():
        if token['ID'] == '0':
            # On ne compte pas les racines
            continue

        if token['UPOS'] not in upos_directions:
            upos_directions[token['UPOS']] = {'left': 0, 'right': 0}

        if token['HEAD'] < int(token['ID']):
            upos_directions[token['UPOS']]['right'] += 1
        else:
            upos_directions[token['UPOS']]['left'] += 1
upos_directions = {k: v for k, v in sorted(upos_directions.items(), key=lambda item: item[0])}        
print(json.dumps(upos_directions, indent=4))


In [ ]:
# Nous allons maintenant afficher ces informations dans un graphique en barres
# Deux options s'offrent à nous :
# - un graphique en barres empilées
# - un graphique en barres côte à côte
# Nous allons utiliser un graphique en barres côte à côte

import numpy as np
# Séparons les données dans des listes "droite" et "gauche"
labels = list(upos_directions.keys())
left_values = [upos['left'] for upos in upos_directions.values()]
right_values = [upos['right'] for upos in upos_directions.values()]

fig, ax = plt.subplots(figsize=(10, 5))

bar_width = 0.35  # épaisseur des barres
index = np.arange(len(labels))  # position des barres

# Traçage des barres de gauche
left_bars = ax.bar(index, left_values, bar_width, label='Gauche', color='skyblue')

# Traçage des barres de gauche
right_bars = ax.bar(index + bar_width, right_values, bar_width, label='Droite', color='orange')

# Add labels, title, and legend
ax.set_xlabel('étiquettes UPOS')
ax.set_ylabel('Comptes')
ax.set_title('Distribution des directions des dépendances par étiquette UPOS : Gauche vs Droite')
ax.set_xticks(index + bar_width / 2)  # Position x-axis ticks in the center of the two bars
ax.set_xticklabels(labels)
ax.legend()

# Rotate the x-axis labels to 45 degrees
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()  # Adjust layout to fit everything
plt.show()

## Directions syntaxiques en fonction des catégories du discours GOV et DEP

In [ ]:
# Ce graphique est intéressant, mais on voudrait aussi connaitre la direction en fonction de la 
# catégorie du discours du gouverneur (HEAD) et afficher cela dans une matrice de position

# Pour cela, nous allons d'abord faire notre dictionnaire de dictionnaire de dictionnaire
# Le premier niveau de clé sera la catégorie du discours du gouverneur (HEAD)
# Le second niveau de clé sera la catégorie du discours du dépendant (DEP)
# Le troisième niveau de clé sera la direction de la dépendance (left ou right)
# La valeur sera le nombre de dépendances dans cette catégorie et dans cette direction

head_dep_directions = {}

for sentence in sentences:
    for token in sentence['treeJson']['nodesJson'].values():
        if token['HEAD'] == 0:
            # On ne compte pas les cas ou la tête est la racine
            continue
        head_token = sentence['treeJson']['nodesJson'][str(token['HEAD'])]
        head_ID = head_token['ID']
        head_UPOS = head_token['UPOS']

        this_upos = token['UPOS']
        if head_UPOS not in head_dep_directions:
            head_dep_directions[head_UPOS] = {}

        if this_upos not in head_dep_directions[head_UPOS]:
            head_dep_directions[head_UPOS][this_upos] = {'left': 0, 'right': 0}

        if token['HEAD'] < int(token['ID']):
            head_dep_directions[head_UPOS][this_upos]['right'] += 1
        else:
            head_dep_directions[head_UPOS][this_upos]['left'] += 1

# On trie le dictionnaire par ordre alphabétique
head_dep_directions = {k: v for k, v in sorted(head_dep_directions.items(), key=lambda item: item[0])}
# Et on trie les dictionnaires imbriqués par ordre alphabétique
for head_upos in head_dep_directions:
    head_dep_directions[head_upos] = {k: v for k, v in sorted(head_dep_directions[head_upos].items(), key=lambda item: item[0])}


print(json.dumps(head_dep_directions, indent=4))

# Nous allons maintenant afficher ces informations dans une matrice de position
# Pour cela, nous allons utiliser la librairie matplotlib

# Nous allons utiliser une fonction de la librairie matplotlib pour afficher la matrice de position
# Cette fonction prend en paramètre une matrice de position et une liste d'étiquettes
# Elle affiche la matrice de position dans un graphique
# Elle retourne un objet "figure" qui contient le graphique
# Nous allons utiliser cet objet pour ajouter un titre à notre graphique


In [ ]:
# Pour la matricde de position, il nous faut, pour chaque paire de catégories du discours (HEAD, DEP),
# un ratio de gouvernance à droite par rapport au nombre total de gouvernance
# Nous allons donc créer un dictionnaire de dictionnaire de dictionnaire de ratio
# Le premier niveau de clé sera la catégorie du discours du gouverneur (HEAD)
# Le second niveau de clé sera la catégorie du discours du dépendant (DEP)

head_dep_ratios = {}
head_dep_total = {}

for head_upos in head_dep_directions:
    head_dep_ratios[head_upos] = {}
    head_dep_total[head_upos] = {}
    for dep_upos in head_dep_directions[head_upos]:
        left_relation = head_dep_directions[head_upos][dep_upos]['left']
        right_relation = head_dep_directions[head_upos][dep_upos]['right']
        total_relation = left_relation + right_relation
        if total_relation < 10:
            head_dep_ratios[head_upos][dep_upos] = 0
        else:
            head_dep_ratios[head_upos][dep_upos] = (left_relation + 0.0001) / total_relation 
        head_dep_total[head_upos][dep_upos] = total_relation

print(json.dumps(head_dep_ratios, indent=4))
print(json.dumps(head_dep_total, indent=4))

In [ ]:
# Nous allons d'abord afficher le nombre de relation par pair de catégories du discours
# Il est important de faire cela avant d'afficher le ratio, car le ratio ne veut rien dire si le nombre de relation est trop faible

import pandas as pd
import seaborn as sns
# Créer un DataFrame pandas pour stocker les données
upos_df = pd.DataFrame(head_dep_total)
position_matrix = upos_df.groupby(upos_df.columns, axis=1).sum().astype(int)

# Créer la matrice de position avec seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(position_matrix, annot=True, cmap="Blues", fmt='d')

# Ajouter les titres et labels
plt.title('Nombre de dépendances par catégorie du discours du gouverneur (HEAD) et du dépendant (DEP)')
plt.xlabel('Gouverneurs')
plt.ylabel('Dépendants')

# Afficher la matrice de position
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
# Créer un DataFrame pandas pour stocker les données
upos_df = pd.DataFrame(head_dep_ratios)
# Pour les besoins de la visualisation, nous allons sommer les valeurs de 'left' et 'right'
# pour chaque paire d'étiquettes UPOS
position_matrix = upos_df.groupby(upos_df.columns, axis=1).sum()
position_matrix.replace(0, np.nan, inplace=True)

# Créer la matrice de position avec seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(position_matrix, annot=True, cmap="Blues", fmt='.2f')

# Ajouter les titres et labels
plt.title('Ratio de dépendances à droite par rapport au nombre total de dépendances')
plt.xlabel('Gouverneurs')
plt.ylabel('Dépendants')

# Afficher la matrice de position
plt.show()

### Questions :
Dans ce treebank du Naija SUD :
- L'auxiliaire est-t'il plus souvent dépendant ou gouverneur ? 
- Une ponctuation peut-elle gouverner une non-ponctuation ? Arriverez vous à retrouver ces cas dans le treebank ?
- Quelles sont les catégories qui gouvernent le plus ?
- Et quelles sont les catégories qui gouvernent le moins ?
- Quelle est la classe syntaxique qui est la plus contrainte sur son emplacement ?

## Quelques algos supplémentaires

In [ ]:
# Calculons le nombre de nom propre/nom commun par phrase en moyenne dans ce corpus

In [ ]:
# Calculons le nombre de verbe par phrase en moyenne dans ce corpus


In [ ]:
# Quels sont les trigrammes syntaxiques les plus fréquents dans ce corpus ?
# On définit un trigramme comme un ensemble de trois tokens consécutifs, dont deux sont dépendants du troisième

trigrams = {}

for sentence in sentences:
    tokens = list(sentence['treeJson']['nodesJson'].values())
    for i in range(len(tokens) - 2):
        token1 = tokens[i]
        token2 = tokens[i+1]
        token3 = tokens[i+2]

        # on ne compte pas si l'un des tokens est une ponctuation
        if token1['UPOS'] == 'PUNCT' or token2['UPOS'] == 'PUNCT' or token3['UPOS'] == 'PUNCT':
            continue
        # if token1['HEAD'] == token2['HEAD'] == token3['HEAD']:
        #     # On ne compte pas les trigrammes dont les trois tokens ont le même gouverneur
        #     continue
        trigram_form = " , ".join([token1['FORM'], token2['FORM'], token3['FORM']])
        if trigram_form not in trigrams:
            trigrams[trigram_form] = 0
        trigrams[trigram_form] += 1

# sort by decreasing frequency
trigrams = {k: v for k, v in sorted(trigrams.items(), key=lambda item: item[1], reverse=True)}
print(json.dumps(trigrams, indent=4))

In [ ]:
# Ce n'est pas pertinent, on a juste les trigrammes les plus fréquents, mais pas
# les plus dépendants les uns des autres
# Il faut donc diviser par la multiplication des fréquences de chaque token

# On récupère donc les fréquences des tokens
monogrammes = {}

for sentence in sentences:
    for token in sentence['treeJson']['nodesJson'].values():
        if token['UPOS'] == 'PUNCT':
            continue
        if token['FORM'] not in monogrammes:
            monogrammes[token['FORM']] = 0
        monogrammes[token['FORM']] += 1

trigrams_PMI = {}
for trigram in trigrams:
    token1, token2, token3 = trigram.split(" , ")
    trigrams_PMI[trigram] = trigrams[trigram] / (monogrammes[token1] * monogrammes[token2] * monogrammes[token3])

# sort by decreasing frequency
trigrams_PMI = {k: v for k, v in sorted(trigrams.items(), key=lambda item: item[1], reverse=True)}


print(json.dumps(trigrams_PMI, indent=4))

In [ ]:
# Ce n'est encore pas si interessant, puisque nous avons en tête des trigrammes qui n'apparaissent qu'une seule fois
# Nous allons donc filtrer les trigrammes qui apparaisent moins de N fois (par exemple 10 fois)

trigrams_PMI = {}


N = 5 # changez cette valeur pour voir les trigrammes qui apparaissent plus ou moins souvent
# au plus cette valeur sera basse, au plus on aura de trigrammes irréguliers (des "outliers" non représentatifs)
# par exemple, en dessous de N=5, vous verrez beaucoup de noms propres et suite de numéraux

for trigram in list(trigrams.keys()):
    
    if trigrams[trigram] > N:
        token1, token2, token3 = trigram.split(" , ")
        trigrams_PMI[trigram] = trigrams[trigram] / (monogrammes[token1] * monogrammes[token2] * monogrammes[token3])

trigrams_PMI = {k: v for k, v in sorted(trigrams_PMI.items(), key=lambda item: item[1], reverse=True)}
print(json.dumps(trigrams_PMI, indent=4))
print("{} trigrammes ont été trouvés".format(len(trigrams_PMI)))


## Pour aller plus loin

### D'autres concepts interessants
- L'espagnol est une langue pro-drop, c'est à dire que le sujet pronominal d'un verbe peut être effacé. Qu'en est t-il du naija ? Quel est le ratio de verbe sans sujet ? Est-ce que ce ratio seul suffit à proposer une hypothèse ?
- Nous n'avons pas encore traçé les distributions des longueurs de dépendances syntaxique.

### Comparaison 
Vous pouvez trouver dans le dossier `data` d'autres treebanks SUD. On peut donc comparer certains des indicateurs que nous avons calculés ici avec ceux des autres langues.

